In [1]:
import torch
from transformers import BertModel

import os, sys, time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from scipy import stats
from collections import defaultdict
import numpy as np
import pandas as pd
import statistics
import matplotlib.pyplot as plt
#from sklearn.feature_extraction import stop_words
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
import transformers
from ipywidgets import FloatProgress
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import copy
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from collections import OrderedDict
from PIL import Image
plt.ion()   # interactive mode


In [2]:
data_folder=os.getcwd()
test_set="book30-listing-test.csv"
train_set="book30-listing-train.csv"

test=pd.read_csv(os.path.join(data_folder+"/"+test_set), sep=",", encoding='latin_1') #'unicode_escape'
train=pd.read_csv(os.path.join(data_folder+"/"+train_set), sep=",", encoding='latin_1')
#setting up columns' names
test=test.set_axis(["AMAZON INDEX (ASIN)","FILENAME","IMAGE URL","TITLE","AUTHOR","CATEGORY ID","CATEGORY"], axis=1, inplace=False)
train=train.set_axis(["AMAZON INDEX (ASIN)","FILENAME","IMAGE URL","TITLE","AUTHOR","CATEGORY ID","CATEGORY"], axis=1, inplace=False)

#selecting title and category id
test_set = test[['FILENAME','TITLE','CATEGORY ID', "CATEGORY"]]
train_set = train[['FILENAME','TITLE','CATEGORY ID', "CATEGORY"]]
#train_set2 = train_set.sample(frac=1).reset_index(drop=True) #shuffling for val data set
#print(train_set)
#print(train_set2.head())


In [3]:
length = len(train_set) + len(test_set)
##print(len(test_set))
#print(length)
print(0.8*length, 0.1* length, 0.1*length)

train_s=train_set[:45598] #change to train_set2
val_s=train_set[45598:] #change to train_set2

print(f'LENGTH OF VALIDATION SET: {len(val_s)}, \
      LENGTH OF TEST SET: {len(test_set)},  \
      LENGTH OF TRAIN SET: {len(train_s)}. ')


45598.4 5699.8 5699.8
LENGTH OF VALIDATION SET: 5701,       LENGTH OF TEST SET: 5699,        LENGTH OF TRAIN SET: 45598. 


In [4]:
#checking for the longest title -> we want to set padding to max_length instead of 512
length_of_the_messages_test = test_set['TITLE'].str.split("\\s+")
length_of_the_messages_train = train_set['TITLE'].str.split("\\s+")

print(f"Max number of words in test_set: {length_of_the_messages_test.str.len().max()}")
print(f"Max number of words in train_set: {length_of_the_messages_train.str.len().max()}")
print("Index = ", length_of_the_messages_train.str.len().idxmax())

print(length_of_the_messages_train[4984])

Max number of words in test_set: 53
Max number of words in train_set: 74
Index =  4984
['[', 'Antidiabetic', 'Agents:', 'Recent', 'Advances', 'in', 'Their', 'Molecular', 'and', 'Clinical', 'Pharmacology', '[', 'ANTIDIABETIC', 'AGENTS:', 'RECENT', 'ADVANCES', 'IN', 'THEIR', 'MOLECULAR', 'AND', 'CLINICAL', 'PHARMACOLOGY', 'BY', 'Testa', '-.', 'Meyer,', '-.', 'Meyer', '(', 'Author', ')', 'May-28-1996[', 'ANTIDIABETIC', 'AGENTS:', 'RECENT', 'ADVANCES', 'IN', 'THEIR', 'MOLECULAR', 'AND', 'CLINICAL', 'PHARMACOLOGY', '[', 'ANTIDIABETIC', 'AGENTS:', 'RECENT', 'ADVANCES', 'IN', 'THEIR', 'MOLECULAR', 'AND', 'CLINICAL', 'PHARMACOLOGY', 'BY', 'TESTA', '-.', 'MEYER,', '-.', 'MEYER', '(', 'AUTHOR', ')', 'MAY-28-1996', ']', 'By', 'Testa', '-.', 'Meyer,', '-.', 'Meyer', '(', 'Author', ')May-28-1996', 'Hardcover']


In [5]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, X, path, train = False):

        self.labels = [label for label in X['CATEGORY ID']]  #list
        self.texts = [tokenizer(title, truncation=True, padding='max_length', max_length = 74,  #512
                    return_tensors="pt") for title in X['TITLE']]
        self.images_name = [img_name for img_name in X['FILENAME']] 
        self.path = path
        self.label_name = [lbl_name for lbl_name in X['CATEGORY']]
        self.train = train
        
    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]
    
    def get_batch_label_name(self, idx):
        return self.label_name[idx]
            
    def get_batch_images_names(self, idx):
        # Fetch a batch of image names
        return self.images_name[idx]
    
    def get_batch_images(self, idx):
        # Fetch a batch of images
        batch_images_name = self.get_batch_images_names(idx)
        batch_label_name = self.get_batch_label_name(idx)
        img = Image.open(os.path.join(self.path, batch_label_name, batch_images_name)).convert('RGB')
        self.train = train
        
        if train:
            train_transform  =  transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
            img = train_transform(img)
        else:
            val_transform  =  transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
            img = val_transform(img)
                
        return img
        

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        batch_image = self.get_batch_images(idx)
        
        return batch_texts, batch_y, batch_image

In [6]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased',
                                         do_lower_case=True)
length = len(train_set) + len(test_set)
##print(len(test_set))
#print(length)
print(0.8*length, 0.1* length, 0.1*length)

train_s=train_set[:45598] #change to train_set2
val_s=train_set[45598:] #change to train_set2

print(f'LENGTH OF VALIDATION SET: {len(val_s)}, \
      LENGTH OF TEST SET: {len(test_set)},  \
      LENGTH OF TRAIN SET: {len(train_s)}. ')

train, val = Dataset(train_s, "/home/gussikoju@GU.GU.SE/images/train", True), Dataset(val_s, "/home/gussikoju@GU.GU.SE/images/train")

45598.4 5699.8 5699.8
LENGTH OF VALIDATION SET: 5701,       LENGTH OF TEST SET: 5699,        LENGTH OF TRAIN SET: 45598. 


In [7]:
val.__getitem__(2)

({'input_ids': tensor([[  101,  1103,  2377,  3955,   112,   188,  3043,  9506,   131,  1121,
           1148,  3043,  1106,  2029, 10890,   113,  1103,  3043,  9506,  1116,
           1326,   114,   102,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
    

In [21]:
#model_text = BertModel.from_pretrained('bert-base-cased', num_labels=30)

#model_text.load_state_dict(torch.load('best_model.pth')['state_dict'], strict=False)
#model_text.load_state_dict(torch.load('best_model.pth'),strict=False)

model_state_dict = torch.load('best_model.pth')
model_text = BertModel.from_pretrained( 'bert-base-cased', state_dict = model_state_dict, num_labels=30)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['linear.weight', 'linear.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
model_text = BertModel.from_pretrained('bert-base-cased', num_labels=30, output_attentions = False, 
   output_hidden_states = False)

model_text.load_state_dict(torch.load('best_model.pth')['state_dict'], strict=False)
#model_text.load_state_dict(torch.load('best_model.pth'),strict=False)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyError: 'state_dict'

In [13]:
data_dir = './images'

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=8)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.pretrained = torchvision.models.resnet50(pretrained=False)
        self.pretrained.fc = torch.nn.Identity()
        
        self.linear = torch.nn.Linear(2048, 30)
       


    def forward(self, x):        
        x = self.pretrained(x)
        x = self.linear(x)
        return x


In [22]:
model_img = models.resnet50(pretrained=True)
num_ftrs = model_img.fc.in_features
model_img.fc = nn.Sequential(nn.Linear(2048, 512),
                         nn.ReLU(),
                         nn.Dropout(0.2),
                         nn.Linear(512, 30),
                         nn.LogSoftmax(dim=1))
#model_img = CNN()
#model_img = model.cuda()
model_img.load_state_dict(torch.load('CNN_model.pth'))

#pretrained_model = torch.load('CNN_modelFTnew.pth')
#model = models.resnet50(pretrained=False)
#model.load_state_dict(pretrained_model.state_dict(),strict=False)
#pretrained_model.load_state_dict(torch.load('CNN_modelFTnew.pth'),strict=False)


<All keys matched successfully>

In [ ]:
#https://stackoverflow.com/questions/53612835/size-mismatch-for-fc-bias-and-fc-weight-in-pytorch

In [38]:
class MultimodalModel(nn.Module):
    def __init__(self, model_text, model_img):
        super(MultimodalModel, self).__init__()
        self.model_text = model_text
        self.model_img = model_img
        self.classifier = nn.Linear(30+30, 30)
        
    def forward(self, img, text, text_mask):
        
        input_text = self.model_text(text.view(74), text_mask.view(74))
        input_img = self.model_img(img)
       
        input_all = torch.cat((input_text, input_img), dim=1)
        
        x = self.classifier(nn.functional.relu(x))
        
        return x

my_model = MultimodalModel(model_text, model_img)
#my_model = my_model.cuda()

In [39]:
text_new, labels, img= train[16]
tekst, text_mask = text_new['attention_mask'], text_new['input_ids']
#img = img.cuda()
#text_new = text.cuda()
#labels = labels.cuda()

#print(textsss['attention_mask'])
predictions = my_model(img.unsqueeze(0), tekst.unsqueeze(0), text_mask.unsqueeze(0))
#print(labels.shape, labels.size)
print(img.shape, tekst.view(74).shape, text_mask.view(74).shape, labels.shape)

ValueError: not enough values to unpack (expected 2, got 1)

In [36]:
print(tekst.view(74).shape)
print(tekst.view(74))

torch.Size([74])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0])


In [37]:
print(tekst.shape)
print(tekst)

torch.Size([1, 74])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]])
